In [29]:
# !pip install selenium

###Load libraries
#import awscli
#import selenium
#import boto3
import pandas as pd
import time
from selenium import webdriver

## Part I. Create web-scraper to load csv file into S3 Bucket 

In [59]:

####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome(r"C:\Users\11946/chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

table = browser.find_element_by_css_selector('table.Bordered')

#####CREATE DATE FRAME#####
#create empty dataframe
def create_DataFrame(table):

    df =[]

    #loop through dataframe to export table
    for row in table.find_elements_by_css_selector('tr'):
          cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


    #update dataframe with header 
    df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])

    #remove the blank row in the csv output file
    df = df.iloc[1:]
    
    return df

table1 = create_DataFrame(table)

for i in range(1,9):
    if i == 2 or i == 3:
        continue
    inputElement_all = browser.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a["+str(i)+"]").click()
#identify the table to scrape
    table = browser.find_element_by_css_selector('table.Bordered')
    table2 = create_DataFrame(table)
    table1 = table1.append(table2, ignore_index=True)

display(table1)


,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
1,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
2,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
3,#HicksStrong Inc.,48-10-48,842612081,NFP,CLIFTON PARK,NY
4,#WalkAway Foundation,47-15-80,832820906,NFP,CARLSBAD,CA
...,...,...,...,...,...,...
95,University of Virginia Health Foundtion,40-44-88,412097394,NFP,CHARLOTTESVILLE,VA
96,Violin Player,41-40-19,270773158,NFP,EAST AMHERST,NY
97,"William A. Epps Community Center, Inc.",40-91-11,861074714,NFP,STATEN ISLAND,NY
98,WORLD SOCIETY OF CZESTOCHOWA JEWS AND THEIR DE...,40-46-49,205101779,NFP,NEW YORK,NY


In [60]:
table1.EIN.nunique()

100

In [61]:
###LOAD THE FILE INTO S3####
# prepare csv file name   
pathname = 's3:/jinming/'#specify location of s3:/{my-bucket}/
filename= 'all_charities_bureau_scrape_data' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file

#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
table1.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3:/jinming/all_charities_bureau_scrape_data20210408234409.csv
